In [ ]:
# This file is to derive the throughput of EEBED.

# The limitation of qiskit, including 1) qiskit can simulate quantum channel noise but can not the real quantum channel, 2) While the simulation runs, some parallel quantum gates are executed sequentially. 3), it can not run large circuit due to the large resource requirement. 
# Because above limitations, We indirectly derive EEBED's throughput. The time cost of EEBED consists of three parts: the time cost of short distance ED, the time cost of EE(decoder and decoupler), and the time cost of entanglement swapping.

# 1) The time cost of short distance ED: The time cost of short distance ED is calculated from the speed of light and the length of the channel. Assuming that the length of the channel is 50km and the speed of light is 300000km/s, it takes about 0.16 ms for the local domain controller to complete the short distance ED of adjacent Repeaters. In EEBED, a local domain controller must prepare and distribute at least three entangled pairs, which can only be executed sequentially. So the time cost is 3*0.16=0.48ms. Short distance ED can be performed in parallel between different local domain controllers.

# 2) The time cost of EE: EE between Repeaters can be executed parallelly, so we only need to calculate the time cost for one EE, including the time costs of the decoder and decoupler.  We can run the following code to derive it.

# 3) The time cost of entanglement swapping: it can be derived from the ordinary long distance ED.

In [1]:
# This code cell is to define a circuit which include a decoder and a decoupler of EE. 
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi
from qiskit_aer import AerSimulator

qreg_q = QuantumRegister(11, 'q')
creg_c0 = ClassicalRegister(1, 'c0')
creg_c1 = ClassicalRegister(1, 'c1')
creg_c2 = ClassicalRegister(1, 'c2')
circuit = QuantumCircuit(qreg_q, creg_c0, creg_c1, creg_c2)

circuit.h(qreg_q[0])
circuit.x(qreg_q[3])
circuit.x(qreg_q[4])
circuit.x(qreg_q[5])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[2])
circuit.cx(qreg_q[1], qreg_q[9])
circuit.cx(qreg_q[0], qreg_q[10])
circuit.cx(qreg_q[2], qreg_q[8])
circuit.x(qreg_q[0])
circuit.x(qreg_q[1])
circuit.x(qreg_q[2])
circuit.ccx(qreg_q[0], qreg_q[1], qreg_q[3])
circuit.ccx(qreg_q[1], qreg_q[2], qreg_q[4])
circuit.ccx(qreg_q[0], qreg_q[2], qreg_q[5])
circuit.x(qreg_q[0])
circuit.x(qreg_q[1])
circuit.x(qreg_q[2])
circuit.ccx(qreg_q[3], qreg_q[4], qreg_q[6])
circuit.ccx(qreg_q[5], qreg_q[6], qreg_q[7])
circuit.ccx(qreg_q[3], qreg_q[4], qreg_q[6])
circuit.ccx(qreg_q[0], qreg_q[2], qreg_q[5])
circuit.ccx(qreg_q[1], qreg_q[2], qreg_q[4])
circuit.ccx(qreg_q[0], qreg_q[1], qreg_q[3])
circuit.ccx(qreg_q[3], qreg_q[5], qreg_q[0])
circuit.ccx(qreg_q[3], qreg_q[4], qreg_q[1])
circuit.h(qreg_q[0])
circuit.h(qreg_q[1])
circuit.ccx(qreg_q[4], qreg_q[5], qreg_q[2])
circuit.h(qreg_q[2])
circuit.measure(qreg_q[0], creg_c0[0])
circuit.measure(qreg_q[1], creg_c1[0])
circuit.measure(qreg_q[2], creg_c2[0])
circuit.z(qreg_q[7]).c_if(creg_c0, 1)
circuit.z(qreg_q[7]).c_if(creg_c1, 1)
circuit.z(qreg_q[7]).c_if(creg_c2, 1)
#circuit.draw("mpl")

In [6]:
# deriving the time cost of one EE.
sim_ideal = AerSimulator(max_memory_mb=65536)
ee_time = sim_ideal.run(circuit,shots=1).result().time_taken
print(ee_time)


0.0021157264709472656


In [7]:
# calculating the throughput of EE (pairs/s or eps)
es_time_3_nodes = 1.09   # deriving from the ordinary_long_distance_ED
es_time_4_nodes = 1.41

EEBED_total_time_3_nodes = 0.48 + ee_time * 1000 + es_time_3_nodes
EEBED_total_time_4_nodes = 0.48 + ee_time * 1000 + es_time_4_nodes

EEBED_throughput_3_nodes = 1/(EEBED_total_time_3_nodes/100)
EEBED_throughput_4_nodes = 1/(EEBED_total_time_4_nodes/100)

print(EEBED_throughput_3_nodes) 
print(EEBED_throughput_4_nodes)

27.131693246433198
24.9642607215645
